In [1]:
import liana as li
import scanpy as sc
from anndata import AnnData
import pandas as pd
from liana.mt import rank_aggregate
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
from pycirclize import Circos

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# loading original anndata
pvat_24weeks_control_female = sc.read_h5ad('../../exported_data/pvat_24weeks_control_female.h5ad')
pvat_24weeks_control_female

AnnData object with n_obs × n_vars = 7534 × 20743
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells'
    uns: 'cellchat_interaction_table_count', 'cellchat_res', 'cellchat_res_filtered', 

In [3]:
pvat_24weeks_control_female.obs['celltype_broad'].unique().tolist()

['Adipocytes',
 'Endothelial_Cells',
 'Immune_Cells',
 'Pericytes',
 'Fibroblasts',
 'Mesothelial_Cells',
 'Neuronal_Cells',
 'SMCs']

In [4]:
# List available methods in lliana framework
li.mt.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...


In [5]:
pvat_24weeks_control_female.raw = pvat_24weeks_control_female
pvat_24weeks_control_female.raw.X

<7534x20743 sparse matrix of type '<class 'numpy.float64'>'
	with 11084559 stored elements in Compressed Sparse Row format>

In [6]:
# Run CellPhoneDB analysis on PVAT data
cellphonedb(pvat_24weeks_control_female, groupby='celltype_broad', expr_prop=0.1, resource_name='mouseconsensus', verbose=True, key_added='cellphonedb_res')

Using `.raw`!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
1774 features of mat are empty, they will be removed.
Make sure that normalized counts are passed!
/Users/pandavis/miniconda3/lib/python3.11/site-packages/liana/method/_pipe_utils/_pre.py:148: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
['Cyp2c6_v1-ps2'] contain `_`. Consider replacing those!
Using resource `mouseconsensus`.
0.21 of entities in the resource are missing from the data.


Generating ligand-receptor stats for 7534 samples and 18969 features


100%|███████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 286.02it/s]


In [7]:
ligand_receptor_pvat = pvat_24weeks_control_female.uns['cellphonedb_res']
ligand_receptor_pvat.head()

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
2337,Slit3,Slit3,37.883728,1.000000,Robo4,Robo4,0.483237,0.310056,SMCs,Endothelial_Cells,19.183483,0.0
8661,Ncam1,Ncam1,1.000000,0.636364,Cacna1c,Cacna1c,18.441858,0.976744,Neuronal_Cells,SMCs,9.720929,0.0
9206,Ntf3,Ntf3,3.790698,0.813953,Ntrk3,Ntrk3,13.953486,0.906977,SMCs,SMCs,8.872092,0.0
1033,Ntrk3,Ntrk3,13.953486,0.906977,Ptprs,Ptprs,1.408235,0.630271,SMCs,Adipocytes,7.680861,0.0
8138,Ntf3,Ntf3,1.145576,0.429932,Ntrk3,Ntrk3,13.953486,0.906977,Fibroblasts,SMCs,7.549531,0.0


In [8]:
ligand_receptor_string = (ligand_receptor_pvat.loc[:,'ligand_complex'].str.contains('Col')) & (ligand_receptor_pvat.loc[:,'receptor_complex'].str.contains('Itg'))
cell_source_string = ligand_receptor_pvat.loc[:,'source'] == 'Fibroblasts'
cell_target_string = ligand_receptor_pvat.loc[:,'target'] == 'Adipocytes'
cell_pvals = ligand_receptor_pvat.loc[:,'cellphone_pvals'] <= 0.05

ligand_receptor_pvat[ligand_receptor_string & cell_source_string & cell_target_string & cell_pvals]

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals
279,Col4a1,Col4a1,4.808171,0.687075,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,2.681611,0.0
304,Col5a3,Col5a3,3.991845,0.759184,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,2.273448,0.0
310,Col3a1,Col3a1,3.353748,0.744218,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.954399,0.0
273,Col6a6,Col6a6,3.212251,0.617687,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.883651,0.0
285,Col4a2,Col4a2,3.089798,0.650340,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.822424,0.0
361,Col4a2,Col4a2,3.089798,0.650340,Itgb5,Itgb5,0.246864,0.184636,Fibroblasts,Adipocytes,1.668331,0.0
289,Col6a2,Col6a2,2.485716,0.738776,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.520383,0.0
291,Col6a3,Col6a3,1.848974,0.662585,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.202013,0.0
287,Col6a1,Col6a1,1.681628,0.617687,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.118339,0.0
275,Col6a5,Col6a5,1.629929,0.431293,Itgb1,Itga1_Itgb1,0.555051,0.362662,Fibroblasts,Adipocytes,1.092490,0.0


In [9]:
ligand_receptor_pvat.to_csv("pvat_24weeks_control_female_cellphonedb.csv")
pvat_24weeks_control_female.write("../../exported_data/pvat_24weeks_control_female.h5ad")